# Import libraries

In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Activation,Conv2D,MaxPooling2D,add,Dropout,Flatten,Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import os
import cv2
import numpy as np

# Importing Training Data and Validation Data

In [2]:
train_data =[]
train_label = []
valid_data =[]
valid_label = []

train_mask_path = 'dataset/train/with_mask'
train_nomask_path = 'dataset/train/without_mask'
valid_mask_path = 'dataset/valid/with_mask'
valid_nomask_path = 'dataset/valid/without_mask'

train_mask_imgs=os.listdir(train_mask_path)
train_nomask_imgs=os.listdir(train_nomask_path)
valid_mask_imgs=os.listdir(valid_mask_path)
valid_nomask_imgs=os.listdir(valid_nomask_path)

train_mask_cnt = 0
for j in train_mask_imgs:
    train_mask_cnt += 1
    img = cv2.imread(train_mask_path+'/'+j,0)
    train_data.append(img_to_array(img))
    
for k in range(0,train_mask_cnt):
    train_label.append(1)
    
train_nomask_cnt = 0
for j in train_nomask_imgs:
    train_nomask_cnt += 1
    img = cv2.imread(train_nomask_path+'/'+j,0)
    train_data.append(img_to_array(img))
    
for k in range(0,train_nomask_cnt):
    train_label.append(0)
    
valid_mask_cnt = 0
for j in valid_mask_imgs:
    valid_mask_cnt += 1
    img = cv2.imread(valid_mask_path+'/'+j,0)
    valid_data.append(img_to_array(img))

for k in range(0,valid_mask_cnt):
    valid_label.append(1)

valid_nomask_cnt = 0
for j in valid_nomask_imgs:
    valid_nomask_cnt += 1
    img = cv2.imread(valid_nomask_path+'/'+j,0)
    valid_data.append(img_to_array(img))
    
for k in range(0,valid_nomask_cnt):
    valid_label.append(0)

In [3]:
len(train_data),len(valid_data)

(50, 50)

# Reshaping data to feed to our CNN

In [4]:
train_data = np.reshape(train_data, (len(train_data), 224, 224,1))
valid_data = np.reshape(valid_data, (len(valid_data), 224, 224,1))

In [5]:
train_label = np.reshape(train_label, (len(train_label)))
valid_label = np.reshape(valid_label, (len(valid_label)))

In [6]:
train_data.shape,train_label.shape

((50, 224, 224, 1), (50,))

# Creating our CNN model and applying Keras Tuner

In [7]:
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
	conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
	# 5x5 conv
	conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
	conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase, assumes channels last format
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	# add filters, assumes filters/channels last
	layer_out = add([conv2, merge_input])
	# activation function
	layer_out = Activation('relu')(layer_out)
	return layer_out

In [10]:
def build_model(hp):  

    # define model input
    visible = Input(shape=(224, 224, 3))

    layer = Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                   kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), 
                   padding='same', activation='relu', kernel_initializer='he_normal')(visible)
    
    layer = MaxPooling2D(pool_size=hp.Choice('maxpool_1_kernel', values = [3,5]), 
                         strides=(2,2), padding='valid')(layer)
    
    layer = inception_module(layer, 32, 16, 16, 16, 16, 16)
    layer = residual_module(layer,
                           n_filters=hp.Int('res_1_filter', min_value=32, max_value=64, step=16))
    layer = residual_module(layer,
                           n_filters=hp.Int('res_2_filter', min_value=32, max_value=64, step=16))

    layer = Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                   kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                   padding='same', activation='relu', kernel_initializer='he_normal')(layer)
    layer = MaxPooling2D(pool_size=hp.Choice('maxpool_2_kernel', values = [3,5]), 
                         strides=(2,2), padding='valid')(layer)

    layer = Flatten()(layer)
    layer = Dropout(0.5)(layer)
    
    layer = Dense(units=2, activation='softmax')(layer)
    
    model = Model(iSnputs=visible, outputs=layer)


    model.compile(optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [12]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=10)

In [13]:
tuner_search.search_space_summary()

Search space summary
Default search space size: 9
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
maxpool_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
res_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
res_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
maxpool_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}

In [14]:
tuner_search.search(train_data,train_label,epochs=5,validation_data=(valid_data,valid_label))

Trial 10 Complete [00h 00m 52s]
val_accuracy: 0.6399999856948853

Best val_accuracy So Far: 0.6399999856948853
Total elapsed time: 00h 11m 25s
INFO:tensorflow:Oracle triggered exit


In [15]:
tuner_search.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
maxpool_1_kernel: 5
res_1_filter: 32
res_2_filter: 48
conv_2_filter: 48
conv_2_kernel: 3
maxpool_2_kernel: 5
learning_rate: 0.001
Score: 0.6399999856948853
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
maxpool_1_kernel: 5
res_1_filter: 32
res_2_filter: 48
conv_2_filter: 96
conv_2_kernel: 3
maxpool_2_kernel: 3
learning_rate: 0.001
Score: 0.6399999856948853
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 5
maxpool_1_kernel: 5
res_1_filter: 32
res_2_filter: 48
conv_2_filter: 112
conv_2_kernel: 5
maxpool_2_kernel: 3
learning_rate: 0.001
Score: 0.6200000047683716
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
maxpool_1_kernel: 5
res_1_filter: 32
res_2_filter: 64
conv_2_filter: 64
conv_2_kernel: 5
maxpool_2_kernel: 3
learning_rate: 0.001
Score: 0.62000000476

In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 110, 110, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 110, 110, 16) 528         max_pooling2d[0][0]              
______________________________________________________________________________________________